In [1]:
%pip install 'accelerate>=0.26.0' ipywidgets -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline

device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id = "openai/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, 
    torch_dtype=torch_dtype, 
    low_cpu_mem_usage=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

In [19]:
pipe = pipeline(
    "automatic-speech-recognition",
    model="openai/whisper-large-v2",
    chunk_length_s=2,
    batch_size=16,
    torch_dtype=torch_dtype,
    device=device,
    return_language=True
)

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.17G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/4.29k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Device set to use cuda:0


In [17]:
result = pipe(
    './output_silero_2.mp3',
    return_timestamps=True, 
    return_language=True,
    # generate_kwargs={"language": "ru"},
)

/root/miniconda3/envs/pytorch/lib/python3.9/site-packages/transformers/models/whisper/generation_whisper.py:573: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(


In [ ]:
# !sudo apt update && sudo apt install ffmpeg
%pip install git+https://github.com/openai/whisper.git -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import whisper
import torch

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = whisper.load_model("large-v3-turbo").half().to(device)

/root/miniconda3/envs/pytorch/lib/python3.9/site-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=dev

In [13]:
%%time
from torch.amp import autocast

with autocast(device_type='cuda', dtype=torch.float16):
    result = model.transcribe('./output.mp3')

result["text"], result['language']

CPU times: user 2.36 s, sys: 117 ms, total: 2.47 s
Wall time: 1.82 s


(' Artificial intelligence is revolutionizing the way we interact with technology. It has the potential to enhance our daily lives by providing personalized experiences and improving efficiency in various industries. As we continue to innovate, it is essential to consider the ethical implications of AI and ensure that it benefits everyone. Together, we can shape a future where technology and humanity coexist harmoniously.',
 'en')

In [12]:
result['language']

'en'